# Localization Files Synchronizer

This notebook synchronizes variables between `app_localizations_en.dart` and `app_localizations.dart` files in a Flutter project. It identifies missing variables in the main localization file and adds them automatically.

## Import Required Libraries

Import necessary libraries such as os, re, and json for file handling and parsing.

In [ ]:
import os
import re
import json
import datetime

## Load Localization Files

Read the contents of app_localizations_en.dart and app_localizations.dart into memory.

In [ ]:
# Define the paths to the localization files
# You may need to modify these paths based on your project structure
project_dir = r"c:\Users\wailik\Documents\Code\Flutter\demo\demo"
en_file_path = os.path.join(project_dir, "lib", "l10n", "app_localizations_en.dart")
base_file_path = os.path.join(project_dir, "lib", "l10n", "app_localizations.dart")

# Function to read file content
def read_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")
        return None
    except Exception as e:
        print(f"Error reading file {file_path}: {str(e)}")
        return None

# Read the files
en_file_content = read_file(en_file_path)
base_file_content = read_file(base_file_path)

if en_file_content and base_file_content:
    print("Files loaded successfully")
    # Display the first few lines of each file
    print("\nFirst few lines of app_localizations_en.dart:")
    print("\n".join(en_file_content.split("\n")[:10]))
    print("\nFirst few lines of app_localizations.dart:")
    print("\n".join(base_file_content.split("\n")[:10]))
else:
    print("Failed to load one or both files")

## Parse Variables in `app_localizations_en.dart`

Extract all variable names from app_localizations_en.dart using regular expressions.

In [ ]:
def extract_variables(content):
    """Extract variable definitions from localization files."""
    if not content:
        return []
    
    # Pattern to match variable declarations in a localization class
    # This pattern looks for lines like: String get variableName => 'value';
    pattern = r'String\s+get\s+(\w+)\s*=>'
    
    # Find all matches
    variables = re.findall(pattern, content)
    
    return variables

# Extract variables from the English localization file
en_variables = extract_variables(en_file_content)

print(f"Found {len(en_variables)} variables in app_localizations_en.dart")
print("First 10 variables:")
for var in en_variables[:10]:
    print(f"- {var}")

## Compare with `app_localizations.dart`

Identify variables present in app_localizations_en.dart but missing in app_localizations.dart.

In [ ]:
# Extract variables from the base localization file
base_variables = extract_variables(base_file_content)

print(f"Found {len(base_variables)} variables in app_localizations.dart")

# Find variables that are in the English file but missing in the base file
missing_variables = [var for var in en_variables if var not in base_variables]

print(f"\nFound {len(missing_variables)} variables missing in app_localizations.dart:")
for var in missing_variables:
    print(f"- {var}")

## Synchronize Missing Variables

Add missing variables from app_localizations_en.dart to app_localizations.dart and save the updated file.

In [ ]:
def extract_variable_definitions(content, variable_names):
    """Extract complete variable definitions for specified variable names."""
    if not content or not variable_names:
        return {}
    
    variable_definitions = {}
    
    for var_name in variable_names:
        # Pattern to match the complete variable definition
        # This looks for: String get varName => 'value';
        pattern = rf'String\s+get\s+{var_name}\s*=>\s*[^;]+;'
        match = re.search(pattern, content)
        
        if match:
            variable_definitions[var_name] = match.group(0)
    
    return variable_definitions

def add_variables_to_file(file_content, variable_definitions):
    """Add missing variable definitions to the file content."""
    if not file_content or not variable_definitions:
        return file_content
    
    # Find the position where the class definition ends
    # Look for the last closing brace
    last_brace_pos = file_content.rstrip().rfind('}')
    
    if last_brace_pos == -1:
        print("Could not find the class closing brace in the file.")
        return file_content
    
    # Insert the new variable definitions before the last closing brace
    updated_content = (
        file_content[:last_brace_pos] + 
        "\n  // Added automatically by synchronization script on " + 
        datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n" +
        "".join([f"  {definition}\n" for definition in variable_definitions.values()]) +
        file_content[last_brace_pos:]
    )
    
    return updated_content

# Only proceed if there are missing variables
if missing_variables:
    # Extract the complete variable definitions from the English file
    missing_var_definitions = extract_variable_definitions(en_file_content, missing_variables)
    
    # Add these definitions to the base file
    updated_base_content = add_variables_to_file(base_file_content, missing_var_definitions)
    
    # Create a backup of the original file
    backup_file_path = base_file_path + ".bak"
    try:
        with open(backup_file_path, 'w', encoding='utf-8') as file:
            file.write(base_file_content)
        print(f"Created backup at {backup_file_path}")
    except Exception as e:
        print(f"Error creating backup: {str(e)}")
    
    # Save the updated content to the base file
    try:
        with open(base_file_path, 'w', encoding='utf-8') as file:
            file.write(updated_base_content)
        print(f"Successfully updated {base_file_path} with {len(missing_variables)} new variables")
    except Exception as e:
        print(f"Error saving updated file: {str(e)}")
else:
    print("No missing variables to add.")

## Verification

Verify that the synchronization was successful by checking that all variables from the English file are now present in the base file.

In [ ]:
# If we made changes, reload the base file to verify
if missing_variables:
    updated_base_content = read_file(base_file_path)
    updated_base_variables = extract_variables(updated_base_content)
    
    # Check if all English variables are now in the base file
    still_missing = [var for var in en_variables if var not in updated_base_variables]
    
    if still_missing:
        print(f"Warning: {len(still_missing)} variables are still missing after synchronization:")
        for var in still_missing:
            print(f"- {var}")
    else:
        print("Verification successful! All variables from app_localizations_en.dart are now in app_localizations.dart")
        
    # Print stats
    print(f"\nStats after synchronization:")
    print(f"- Variables in app_localizations_en.dart: {len(en_variables)}")
    print(f"- Variables in app_localizations.dart: {len(updated_base_variables)}")